## Investigating Concentrated Liquidity and Impermanent Loss

### Goals
- Look at various ranges of price for each token across time and fee collection
- Want to understand how much fees need to be collected to cover the impermanent loss
- Try to figure out the best ranges based on estimates of total price changes

### Questions
- Should the program be able to apply re-balance using geometric mean?
- Should linear regression be used to help find the answers?  Or just look at 3D plots?
- Can look at both compounding and just collecting fees


### Notes
- Don't bother trying to calculate the fees generated based on volume, just figure out what the fees need to be based on the "final" price of the tokens
- Create "high-side" and "low-side" outs and figure out the fees required and and what the APR would need to average given a timeframe that it did stay in the range 
- Then can get fancy and do re-balances, etc.



In [ ]:
# Look at impermanent loss with a 10% increase 
import helper_classes as hc

seed_usd = 10000
btc = hc.Token("BTC", 97500)
usdc = hc.Token("USDC", 1)

lp = hc.LiquidityPool(btc, usdc)
lp.create_range(seed_usd, 10, 10)
btc.price *= 1.1
lp.update_token_balances(30)
print(lp)


In [ ]:
# Look at IL with a 30% increase with re-balancing to chase it upwards
import helper_classes as hc

seed_usd = 10000
btc = hc.Token("BTC", 97500)
usdc = hc.Token("USDC", 1)

lp = hc.LiquidityPool(btc, usdc)
lp.create_range(seed_usd, 10, 10)
price = [1, 1, -1, -1, -1, 1]
for i in price:
    btc.price *= (1 + i/10)
    lp.update_token_balances(30)
    lp.rebalance(10, 10)
print(lp)

In [ ]:
# Loop LP price and duration and check APR required
import numpy as np
import pandas as pd
import helper_classes as hc
import matplotlib.pyplot as plt

seed_usd = 10000
btc = hc.Token("BTC", 97500)
usdc = hc.Token("USDC", 1)
lp = hc.LiquidityPool(btc, usdc)
lp.create_range(seed_usd, 10, 10)
aprs = []

for price in np.arange(90000, 110000, 1000):
    for duration in np.arange(0, 70, 7):
        btc.price = price
        lp.update_token_balances(duration)
        data = [price, duration, lp.apr]
        aprs.append(data)

data = pd.DataFrame(aprs, columns=['price', 'duration', 'apr'])
print(data)

In [ ]:
# Testers
import helper_classes as hc

seed_usd = 10000
btc = hc.Token("BTC", 97500)
usdc = hc.Token("USDC", 1)
eth = hc.Token("ETH", 3600)

lp = hc.LiquidityPool(btc, usdc)
lp.create_range(seed_usd, 10, 15)
print(lp)
lp.token_x.price = 110000
lp.update_token_balances(30)
print(lp)

lp2 = hc.LiquidityPool(eth, btc)
lp2.create_range(seed_usd, 10, 10)
print(lp2)
lp2.token_x.price = 3800
lp2.token_y.price = 110000
lp2.update_token_balances(30)
print(lp2)